In [1]:
import scipy
from scipy.cluster.hierarchy import linkage
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from random import choices, sample
from collections import Counter

In [90]:
def h_cluster(X, max_clusters=None):
    """
    Builds a hierarchical clustering using the linkage function from scipy 
    (agglomerative clustering using ward's linkage and euclidean distance).
    Records clusters as they merge; output clusters start at the root node.
    The two child nodes, associated with each new node, are recorded for reference.

    Args:
        X (np.ndarray): [N x d] matrix, containing N observations of d-dimensional input data
        max_clusters (int, optional): Determines the maximum number of clusters defined. 
            Default value (N-1) considers clusters down to single observations.

    Returns:
        list: A list containing N-1 arrays, each represents a node in the hierarchical tree,
            containing the data indices associated with each cluster.
        np.ndarray: A [2 x N-1] matrix, containing the 2 child nodes for each cluster.
    """

    print("len of X is ")
    print(len(X))

    
    if max_clusters is None:
        max_clusters = len(X) - 1

    # Build hierarchical tree
    Z = linkage(X, method='ward', metric='euclidean')

    N = len(X)
    u = [None] * (N - 1)
    ch = np.zeros((2, N - 1), dtype=int)

    # Using the output of linkage (Z) to build clusters
    for m in range(len(u)):
        # Join parts p1 & p2 of the clustering together using Z info
        if Z[m, 0] < N:
            p1 = [Z[m, 0]]
            c1 = int(p1[0])
        else:
            p1 = u[int(Z[m, 0] - N)]
            c1 = 2*N - int(Z[m, 0]) - 1

        if Z[m, 1] < N:
            p2 = [Z[m, 1]]
            c2 = int(p2[0])
        else:
            p2 = u[int(Z[m, 1] - N)]
            c2 = 2*N - int(Z[m, 1]) - 1
     
        u[m] = p1 + p2
        ch[:, m] = [c1, c2]

    u = list(reversed(u))
    u = u[:max_clusters]
    u_= [[int(x) for x in sublist] for sublist in u]
    u = [x for x in u_ if x]
    print("first elemnt of u has length ")
    print(len(u[0]))

    ch = np.fliplr(ch[:, :max_clusters])
    ch = ch.astype(int)
    
    return u, ch

In [77]:
def prune_label(P, u, Au, Lu, ch, z, N, eu, wu):
    """
    Function to refine the pruning (P) and labelling (L) for the DH active learner.
    
    Parameters:
    - P : list
        Input pruning, to be refined (array of node/cluster numbers).
    - u : dict
        Contains N-1 arrays. Each array represents a node in the hierarchical tree.
    - Au : list
        Logistic array, indicating the admissibility of each cluster in the current pruning.
    - Lu : dict
        Cell containing the majority label(s), for each node.
    - ch : numpy array
        2x(N-1) matrix containing the 2 children nodes for each cluster.
    - z : numpy array
        Sampled data information array.
    - N : int
        Total number of clusters.
    - eu : list
        Error associated with propagating the majority label to unlabelled instances.
    - wu : list
        Node weights - proportion of the total data in each node.
        
    Returns:
    - P_ : list
        Output (refined) pruning.
    - L : list
        The majority label for each cluster in the refined pruning.
    - XL : numpy array
        The labelled dataset provided by the DH learner.
    """
    
    L = [0] * len(u)  # Initialize admissible cluster label pairs
    P_ = list(P)  # Define working pruning
    print("length of u is ")
    print(len(u))


    for i, v in enumerate(P):  # For each node in the current pruning
        # LABEL parent node in case descendants are not admissible
        Lu[1] = 0  # Arbitrary label of root
        if len(P) != 1 and len(Lu[v]) == 1:
            L[v - 1] = Lu[v]

        # Identify first descendants...
        chv = ch[:, v - 1]
        Pv = [v]
        print("len of au is ")
        print(len(Au))
        print("len of chv is ")
        print(len(chv))
        Achv = [Au[x - 1] for x in chv]

        # While at least one pair of siblings is admissible, refine and label Pv
        while np.sum(np.sum(Achv) == 2) >= 1:
            i_ch = [idx for idx, val in enumerate(Achv) if np.sum(val) == 2]

            for ich in i_ch:
                ep = eu[Pv[ich] - 1]
                ech = (1 / np.sum([wu[x - 1] for x in chv[:, ich]])) * np.sum(
                    [wu[x - 1] * eu[x - 1] for x in chv[:, ich]])
                Lch = [Lu[x] for x in chv[:, ich]]
                Lch_log = [len(x) == 1 for x in Lch]

                if len(Lu[Pv[ich]]) == 1 and ech < ep and np.sum(Lch_log) == 2:
                    Pv[ich] = 0
                    u_ = chv[:, ich]
                    Pv.extend(list(u_))

            Pv = [x for x in Pv if x != 0]

            if len(set(Pv)) > chv.shape[1]:
                chv = ch[:, [x - 1 for x in Pv]]
                Achv = [Au[x - 1] for x in chv.flatten()]
            else:
                break

        if len(Pv) > 1:
            print(f"\nTOTAL CLUSTERS {len(P_)}: nodes {Pv} replace node [{v}]")
            P_ = [x for x in P_ if x != v]
            P_.extend(Pv)
            for uw in Pv:
                L[uw - 1] = Lu[uw]

    xl = [0] * N
    for idx, v in enumerate(P):
        print("z len is ")
        print(len(z))
        if len(P) >= len(set(z[:, -1])):
            xi = u[v]
            xl[xi[0]] = L[v - 1]

    for idx in range(z.shape[0]):
        z_ = int(z[idx, 0])
        zl = z[idx, -1]
        xl[z_ - 1] = zl

    XL = np.array([[idx + 1, x] for idx, x in enumerate(xl) if x != 0])

    return P_, L, XL

In [93]:
def DH_AL(u, ch, B, T, y):
    # Initialize variables
    N = len(u[0])
    print("N is ")
    print(N)
    Nu = [len(ue) for ue in u]
    wu = np.array(Nu) / N
    
    z = np.empty((0, 3))
    uz = [ [] for _ in range(N-1)]
    u_ = u.copy()
    print("u is as: ")
    print(u_)
    
    pl = [None] * len(u)
    Aul = [None] * len(u)
    Au = np.zeros(len(u) + N)
    eu = np.ones(len(u) + N)
    Lu = [None] * len(u)
    
    P = [0]
    
    for t in range(1, T+1):
        for b in range(1, B+1):
            # Select v from P
            prop = wu[P]
            for i, v in enumerate(P):
                coeff = 1 if len(P) == 1 else 0 if not u_[v] else 1 - max(pl[v][:, -2])
                prop[i] *= coeff
            
            prob = prop / np.sum(prop)
            vi = choices(range(len(prob)), prob)[0]
            
            # Query label
            print("P[vi] is ")
            print(P[vi])
            print("and u_ at that value is ")
            print(u_[P[vi]])
            print("vi is ")
            print(vi)
            print("P is ")
            print(P)
            s = 0
            if len(u_[P[vi]]) != 0:
                s = int(sample(u_[P[vi]], 1)[0])
            else:
                continue
            for ui in u_:
                if s in ui:
                    ui.remove(s)
            
            print("s:", s, "Type of s:", type(s))  # Debug: check what s holds
            l = y[s]
            
            sampled_vec = np.array([s, P[vi], l])
            z = np.vstack([z, sampled_vec])
            
            # Update node counts
            u_i = [i for i, ue in enumerate(u) if s in ue]
            for uw in u_i:
                uz[uw].append((s, l))
                nu = len(uz[uw])
                
                cl, c = zip(*Counter([l for s, l in uz[uw]]).items())
                p_l = np.array(c) / nu
                
                delta = 1 / nu + np.sqrt((p_l * (1 - p_l)) / nu)
                lb = np.maximum(p_l - delta, 0)
                ub = np.minimum(p_l + delta, 1)
                
                pl[uw] = np.column_stack((cl, p_l, lb, ub))
                Lu[uw] = cl[np.argmax(p_l)]
                
        # Update admissibilities, error/scores
        u_i = [i for i, ple in enumerate(pl) if ple is not None]
        for uw in u_i:
            beta = 1.5
            p_l = pl[uw]
            if len(uz[uw]) > 1:
                LHS = p_l[:, -2]
                RHS = beta * p_l[:, -1] - 1
                a_l = LHS[:, None] > RHS
                
                a_l[np.diag_indices_from(a_l)] = False
                idx = np.all(a_l, axis=1)
                
                Aul[uw] = p_l[idx, 0]
                Au[uw] = np.any(idx)
                eu[uw] = 1 - np.max(p_l[:, 1]) if np.any(idx) else 1
                
        eu[Au == 0] = 1
        
        # Refine pruning & labelling
        P, L, XL = prune_label(P, u, Au, Lu, ch, z, N, eu, wu)
    
    return XL, np.array(z), P, L

In [5]:
"""
example run from https://github.com/labull/EngineeringPatternRecognition/tree/main/matlab/cluster_based_active_learning
n = 100;
B = 3; % batch size
t = n/3; % number of runs

% run the DH learner
[xl, z] = DH_AL(u, ch, B, t, Y);
"""

'\nexample run from https://github.com/labull/EngineeringPatternRecognition/tree/main/matlab/cluster_based_active_learning\nn = 100;\nB = 3; % batch size\nt = n/3; % number of runs\n\n% run the DH learner\n[xl, z] = DH_AL(u, ch, B, t, Y);\n'

In [26]:
from reader import fetch_signals
from preprocess import preprocess
from feature_extractor import *
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# gets X_train, y_train or X_test, y_test (pca and std should be None if train=True)
def get_train_test(pca = None, scaler = None, train = True):  
    print("benchmark 0")  
    signals, labels = fetch_signals('C:\\Users\\amroa\\Documents\\thesis\\data', train = train)
    print("benchmark 1")
    preprocessed_sig = [preprocess(signal) for signal in signals]
    print("benchmark 2")

    # extract the characteristics
    X_train_time =  np.apply_along_axis(time_domain_characteristics, axis = 1, arr = preprocessed_sig) 
    print("benchmark 3")
    X_train_freq = np.apply_along_axis(frequency_domain_characteristics, axis = 1, arr = preprocessed_sig) 
    print("benchmark 4")
    Y = np.array(labels) - 1

    # unstd stands for unstandardized
    X_unstd = np.hstack((X_train_time, X_train_freq))
    mask_finite = np.isfinite(X_unstd).all(axis=1)
    X_unstd = X_unstd[mask_finite]
    Y = Y[mask_finite]

    # standardize the data and reduce dimensionality using PCA
    if ((scaler  == None) or (pca == None)):
        scaler = StandardScaler()
        X_std = scaler.fit_transform(X_unstd)
        pca = PCA(n_components=0.9, svd_solver = 'full') 
        X = pca.fit_transform(X_std)
        return X, Y, pca, scaler
    else:
        X_std = scaler.transform(X_unstd)
        X = pca.transform(X_std)
        return X, Y, pca, scaler

    return None, None, None, None



In [27]:
X, Y, pca, scaler = get_train_test()
x_test, y_test, _ , _ = get_train_test(pca = pca, scaler = scaler, train = False)  

benchmark 0
benchmark 1
benchmark 2
benchmark 3
benchmark 4
benchmark 0
benchmark 1
benchmark 2
benchmark 3
benchmark 4


In [29]:
set(Y)
print(x_test.shape)
print(y_test.shape)

(60718, 715)
(60718,)


In [30]:
# Random sampling strategy (makes sure all *different* labels are included, plus whatever is needed to fill the algorithm run budget of t*B)
def sample_indices_covering_all_Y_values(Y, N):
    unique_values = np.unique(Y)
    sampled_indices = []
    
    if N < len(unique_values):
        raise ValueError(f"N must be greater than or equal to the number of unique values in Y ({len(unique_values)}).")
        
    for value in unique_values:
        possible_indices = np.where(Y == value)[0]
        sampled_index = np.random.choice(possible_indices)
        sampled_indices.append(sampled_index)
        
    # calculate the remaining number of indices needed
    remaining = N - len(unique_values)
    
    # exclude already sampled indices
    all_indices = np.arange(len(Y))
    remaining_indices = np.setdiff1d(all_indices, sampled_indices)
    
    # sample additional random indices to meet the required N
    additional_indices = np.random.choice(remaining_indices, size=remaining, replace=False)
    
    # combine original sampled indices with the additional random indices
    final_sampled_indices = np.concatenate([sampled_indices, additional_indices])
    
    return final_sampled_indices

In [94]:
from predict import predict, binarize

T = 30  # maximum number of runs (max label budget T*B = 600)
B = 20  # batch size
reps = 20  # number of repeats for each experiment

# For storing error
e_rs = []
""""
for t in range(1, T + 1):
    print(f'\nQUERY BUDGET ------ {t * B}')
    acc = []  # accuracy for each repeat
    for r in range(reps):
        # define the training-set by a random sample from available data
        train_idx = sample_indices_covering_all_Y_values(Y, t*B) #np.random.choice(X.shape[0], t * B, replace=False)
        x_train = X[train_idx, :]
        y_train = Y[train_idx]
        print(x_test.shape)
        print(y_test.shape)

        # CLASSIFICATION
        y_pred = predict(x_train, y_train, x_test, 1)
        acc.append(np.sum(y_pred == y_test) / len(y_test))

    e_rs.append(1 - np.mean(acc))

"""

# CLUSTER BASED ACTIVE LEARNING: the DH learner for an increasing label budget
e_al = []
for t in range(1, T + 1):
    print(f'\nQUERY BUDGET ------ {t * B}')
    acc = []
    for r in range(reps):
        # INITIAL CLUSTERING
        u, ch = h_cluster(X, max_clusters=100)

        # HIERARCHICAL SAMPLING FOR ACTIVE LEARNING
        xl, z, p, l = DH_AL(u, ch, B, t, Y)
        print("********** done ****************")
        train_idx = xl[:, 0].astype(int)  # convert to int for indexing
        x_train = X[train_idx, :]
        y_train = xl[:, 1]

        # CLASSIFICATION
        y_pred = predict(x_train, y_train, x_test, 1)
        acc.append(np.sum(y_pred == binarize(y_test)) / len(y_test))

    e_al.append(1 - np.mean(acc))

# PLOT
plt.figure(3)
plt.plot(np.arange(B, B * T + 1, B), e_rs, '--')
plt.plot(np.arange(B, B * T + 1, B), e_al, '--')
plt.xlabel('label budget (n)')
plt.ylabel('classification error (e)')
plt.legend(['passive learning', 'active learning'])
plt.show()



QUERY BUDGET ------ 20
len of X is 
17881
first elemnt of u has length 
17881
N is 
17881
u is as: 
[[6690, 6737, 6684, 6731, 6696, 6743, 6693, 6740, 6688, 6735, 6685, 6732, 6695, 6742, 6687, 6734, 6691, 6738, 6686, 6733, 6692, 6739, 6682, 6729, 6694, 6741, 6683, 6730, 7216, 7264, 7217, 7265, 7213, 7261, 7221, 7269, 7220, 7268, 7208, 7256, 7215, 7263, 7214, 7262, 7212, 7260, 7210, 7258, 7207, 7255, 16390, 17053, 10737, 10735, 14946, 3381, 3373, 3379, 11792, 10736, 14957, 3374, 14950, 10731, 16002, 613, 755, 237, 232, 6533, 8629, 11796, 8620, 9014, 10733, 10732, 11799, 11786, 236, 17064, 13905, 13892, 13904, 8635, 11800, 8633, 12848, 12847, 17055, 15334, 15478, 15340, 15484, 14288, 14431, 14278, 14421, 14276, 14419, 14285, 14428, 13901, 14955, 11790, 12836, 12838, 17066, 14284, 17062, 17054, 11795, 11797, 10599, 10742, 10982, 3383, 12174, 12318, 11798, 16005, 8632, 12837, 8622, 12849, 8634, 11789, 14954, 16010, 8628, 11785, 11794, 17061, 10729, 13893, 13897, 16003, 6390, 6534, 6914, 70

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12], got [ 1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 14. 16.]

In [95]:
set(Y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [ ]:

import matlab.engine
import numpy as np

# Start a MATLAB engine session
eng = matlab.engine.start_matlab()

# Execute MATLAB code that generates an array
matlab_code = """
function [XL, z, P, L] = DH_AL(u, ch, B, T, y)

N = length(u{1});
% node weights
Nu = cellfun(@(x) size(x,1), u, 'UniformOutput', false); % data count in each node
wu = cell2mat(Nu)./N;

% store queries
z = []; % queries
uz = cell(1,N-1); % quries according to node
u_ = u; % working version of u (sample pool) to prevent resampling

% initialise node properties
pl = cell(1,length(u)); % proportions of each label, with bounds appended
Aul = cell(1,length(u)); % admissible label sets
Au = zeros(1,length(u)+N); % admissible label logistic array
eu = ones(1,length(u)+N); % errors
Lu = cell(1,length(u)); % empty node majority label cell

P = [1]; %#ok<NBRAK> % set pruning to be the first node only

for t = 1:T
%     fprintf('\nRUN%d: QUERYING DATA POINT:', t); % VERBOSE1
%     fprintf('\nRUN%d: PRUNING: \t %s', t, mat2str(P)); % VERBOSE3
    for b = 1:B
        % ------ SELECT v from P ------ %              
        prop = wu(P); % set proportionality to node weight
        for i = 1:numel(P)
            v = P(i); % cluster index label
            if numel(P)==1 % if its the root node prop = 1 
                coeff = 1;
            elseif isempty(u_{v}) % if no more samples remain set prop = 0
                coeff = 0;              
            elseif ~isempty(u_{v}) % THE SELECT RULE
                coeff = 1 - max(pl{v}(:,end-1)); % coeff. prop. to error in that node
            end
            prop(i) = prop(i)*coeff; % update proportionality to include error coefficient
        end
        
        % select node according to proportionality
        prob = prop/norm(prop,1); % nomalise porportionality to make probability
        cs = cumsum(prob); i = 1:length(prob);
        vi = i(1+sum(cs(end)*rand>cs)); % select a cluster index according tp prob.
                
        % ------ QUERY LABEL ------ % 
        % pick a random query index fom the cluster vi in P
        s = datasample(u_{P(vi)},1);
            
        % remove z from the sample pool, u_, to prevent resempling
        d_i = cellfun(@(x) find(x==s), u_, 'UniformOutput',false);
        for i = 1:numel(u_)
            u_{i}(d_i{i})=[];
        end
        
        l = y(s); % query from hidden labels (y)
%         fprintf(' [%d]',s); % VERBOSE2
        z = [z; s, P(vi), l]; %#ok<AGROW> store the point index, the cluster it was sampled from and it's label

        % ------ UPDATE NODE COUNTS ------ % 
        % for all nodes that contain the new sample (bottom up): porportions & bounds
        % determine wich new node contain index s
        log = cellfun(@(x) ismember(s,x), u, 'UniformOutput',false); % logistic array
        u_i = find(cell2mat(log)==1); % cluster numbers to update, contiaining sample s

        for i = 1:numel(u_i)
            uw = u_i(i); % working cluster number  
            uz{uw} = [uz{uw};s,l]; % organise quried points according to nodes
            nu = size(uz{uw},1); % sample counts in each node

            cl = unique(uz{uw}(:,end)); % classes obsrved in that cluster
            c = sum(uz{uw}(:,end)==cl'); % counts for each of the observed classes
            p_l = (c'./nu); % proportions of each label

            % find the bounds of these estimates
            delta = 1/nu + sqrt((p_l.*(1-p_l))/nu); % confidence interval
            lb = max(p_l-delta,0);
            ub = min(p_l+delta,1);

            pl{uw} = [cl,p_l,lb,ub]; % [label, pl (label count/total count for each node)]
            
            % majority label
            Lu{uw} = pl{uw}(pl{uw}(:,2) == max(pl{uw}(:,2)), 1);           
        end   
    end
    
    % ------ UPDATE ADMISSIBILITIES, ERROR/SCORES ------ %
    % after each batch size, B
    % update admissibilities and error for the nodes containing samples
    u_i = find(cell2mat(cellfun(@isempty, pl, 'UniformOutput', false))==0); % vector containing cluster numbers
    
    for i = 1:numel(u_i)
        uw = u_i(i); % working node number
        % BETA HYPERPARAMETER
        beta = 1.5; 
        p_l = pl{uw}; % extract proportion infromation for the working cluster
        if size(uz{uw},1)>1 % if there is more than one sample check if it's asmissible
            % admissibility
            LHS = p_l(:,end-1);
            RHS = (beta.*p_l(:,end)-1);
            a_l = LHS' > RHS; % logical comparison matrix for inequality results, label L compared to all others
            a_l = a_l.*abs(eye(size(a_l))-1); % delete diagonal so that a label is not considered admissible against itself

            idx = find(sum(a_l) == size(p_l,1) - 1); % asmissible labels succeed the inequality for all other labels...       
            Aul{1,uw} = p_l(idx,1); % store labels in the admiss. cell.
            Au(uw) = ~isempty(idx); % mark admissible nodes in Au cell with a [1]
            % error/score
            eu(uw) = (1 - max(p_l(:,2))); % record error for assigning majority label       
            if isempty(idx) 
                eu(uw) = 1; % where there are no admissible labels, overwrite error = 1
            end
        end       
    end
    % adjust the error for undersampled nodes
    eu(Au==0)=1; % where labels are not admissible, the error==1
    
    % ------ REFINE PRUNING & LABELLING ------ %
    % output pruning labelled same as the input, to re-assign 'P'
    % P becomes the refined set of nodes
    [P, L, XL] = prune_label(P, u, Au, Lu, ch, z, N, eu, wu);
end
end
"""
eng.eval(matlab_code, nargout=0)

# Retrieve the MATLAB array and convert it to a Python list
matlab_array = eng.workspace['array']
python_list = list(matlab_array)

# Convert the Python list to a NumPy array
numpy_array = np.array(python_list)

# Print the NumPy array
print(numpy_array)

# Close the MATLAB engine session
eng.quit()
